In [73]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
from deap import benchmarks
import random

In [74]:
def evaluate_rasrigin(individual):
    
    pi=3.141592653589793238
    
    A=10*len(individual)
    
    for i in range(len(individual)):
        
        A+=individual[i]*individual[i]-10.0*np.cos(2.0*pi*individual[i])
       
        
        
    return [A]

In [75]:
def gen():
    
    A=5.12*2.0*(np.random.sample()-0.5)
        
    return A

In [76]:
from deap import creator, base, tools, algorithms

In [77]:
import time

In [78]:
creator.create("FitnessMin", base.Fitness, weights=[-1.0])
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("gen", gen)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.gen, n=5)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)



toolbox.register("evaluate", evaluate_rasrigin)

toolbox.register("mate", tools.cxBlend, alpha=0.2)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)


In [80]:
start=time.process_time()

pop = toolbox.population(n=1000)

CXPB, MUTPB, NGEN = 0.7, 0.1, 150

# Evaluate the entire population
fitnesses = map(toolbox.evaluate, pop)
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

mean_old=0

stop_reason=False
g=0

while (g <NGEN) and (stop_reason==False):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring
    
    fits = [ind.fitness.values[0] for ind in pop]
        
    length = len(pop)
    mean_new = sum(fits) / length
    
    min_new=min(fits);
    
    if mean_new==mean_old:
        print('stop')
        stop_reason=True
        
    mean_old=sum(fits) / length
    g+=1

finish=time.process_time()